In [134]:
from selenium import webdriver
from urllib.request import urlretrieve
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import re
import csv

In [157]:
class mySelenium():
    
    def __init__(self,driver_path, url, isHeadless = False):
        self.url = 'https://'+ url
        self.driver_path = driver_path
        self.isHeadless = isHeadless
        self.driver = None
    
    def init_and_get_chrome_selenium_driver(self):
        
        options = None
        if self.isHeadless is True:
            options = webdriver.ChromeOptions('headless')
            
        self.driver = webdriver.Chrome(executable_path = self.driver_path, chrome_options = options)
    
    def open_instagram(self):
        self.driver.get(self.url)
        
    def logIn_instagram(self, user_data_path):
        userdata = None
        with open(user_data_path,'r') as csvFile:
            userdata = list(csv.reader(csvFile))
            userdata = sum(userdata, [])
        
        self.driver.implicitly_wait(2)
        self.driver.find_element_by_name('username').send_keys(userdata[0])
        self.driver.find_element_by_name('password').send_keys(userdata[1])
        self.driver.find_element_by_xpath('//*[@id="react-root"]/section/main/article/div[2]/div[1]/div/form/div[4]/button').submit()
        
    def search_on_instagram_by(self,topic):
        self.driver.get(self.url + '/explore/tags/'+topic)
        
    def get_first_article_on_popularList(self):
        self.driver.find_element_by_xpath('//*[@id="react-root"]/section/main/article/div[1]/div/div/div[1]/div[1]/a/div').click()
        WebDriverWait(self.driver,60).until(EC.presence_of_element_located((By.CLASS_NAME,'e1e1d')))
        
        pageSource = self.driver.page_source
        bs = BeautifulSoup(pageSource,'html.parser')
        
        user_name = bs.find('div', {'class':'e1e1d'}).find('a', href = re.compile('^/.')).attrs['href']
        img = bs.find('div', {'class': 'KL4Bh'}).find('img')['src']
        
    
        return user_name,img
        
        

In [94]:
class CatData():
    
    def __init__(self, file_name):
    
        self.file_name = file_name
        self.file_path = './' + self.file_name
        
    def save_cat_data(self,name, img_location):
        with open(self.file_path,'a') as csv_file:
            writer = csv.writer(csv_file)
            writer.writerow([name.strip('/') , img_location])
        
    def get_saved_cat_data(self):
        with open(self.file_path, 'r') as csvFile:
            data = csv.reader(csvFile)
            return list(data)
            

In [168]:
myselenium = mySelenium('./chromedriver','www.instagram.com' )

In [169]:
myselenium.init_and_get_chrome_selenium_driver()

In [170]:
myselenium.open_instagram()

In [171]:
myselenium.logIn_instagram('./data.csv')

In [172]:
myselenium.search_on_instagram_by('britishcat')

In [173]:
name, img = myselenium.get_first_article_on_popularList()

In [174]:
catdata = CatData('catData.csv')

In [175]:
catdata.save_cat_data(name,img)

In [176]:
l = catdata.get_saved_cat_data()

In [177]:
urlretrieve(l[0][1],'cat.jpg')

('cat.jpg', <http.client.HTTPMessage at 0x1100cdf98>)